In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)

In [3]:
df = pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [4]:
df.shape

(6000, 785)

In [5]:
x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]    

In [8]:
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [10]:
class MyNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(num_features, 32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x    

In [11]:
learning_rate = 0.001
num_epochs = 50

In [12]:
model = MyNN(1)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4
                       )

In [13]:
for epoch in range(num_epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # forward pass
    outputs = model(batch_features)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1 , Loss: 0.596718911131223
Epoch: 2 , Loss: 0.31723540554443996
Epoch: 3 , Loss: 0.23371583128968876
Epoch: 4 , Loss: 0.15987975103159746
Epoch: 5 , Loss: 0.1044269027095288
Epoch: 6 , Loss: 0.0883605704937751
Epoch: 7 , Loss: 0.0494283296711122
Epoch: 8 , Loss: 0.05107733111673345
Epoch: 9 , Loss: 0.052103101369769624
Epoch: 10 , Loss: 0.046851905414368955
Epoch: 11 , Loss: 0.04652412720839493
Epoch: 12 , Loss: 0.04844562310220984
Epoch: 13 , Loss: 0.029411300701710084
Epoch: 14 , Loss: 0.03373359240048254
Epoch: 15 , Loss: 0.02534222378570121
Epoch: 16 , Loss: 0.013213826082452822
Epoch: 17 , Loss: 0.005302429937704195
Epoch: 18 , Loss: 0.024312334361651058
Epoch: 19 , Loss: 0.01474961050817607
Epoch: 20 , Loss: 0.034539823783416065
Epoch: 21 , Loss: 0.045692067523050356
Epoch: 22 , Loss: 0.012479241608428613
Epoch: 23 , Loss: 0.02143958115717396
Epoch: 24 , Loss: 0.03423668021830963
Epoch: 25 , Loss: 0.01341779313722024
Epoch: 26 , Loss: 0.01397018035249251
Epoch: 27 , Loss:

In [ ]:
#result
